# Cross-resonance gate between transmon and fluxonium: linear approximation

In this notebook we study the cross-resonance effect between a transmon and a fluxonium in the strong drive regime.

In [1]:
import numpy as np
import scipy.integrate
import time
import qutip as qtp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem, Qubit
from pysqkit.util.phys import temperature_to_thermalenergy
import pysqkit.util.transformations as trf
from pysqkit.util.linalg import get_mat_elem
from pysqkit.solvers.solvkit import integrate
from matplotlib.colors import LogNorm
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers import solvkit
import qutip
import itertools
from functools import partial
import multiprocessing
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import util_tf_cr
import copy
import json
import cmath

from IPython.display import display, Latex

The gate is based on the following driven Hamiltonian of a fluxonium and a transmon capacitively coupled

$$H = H^{(0)}_{T, 1} + H^{(0)}_{T, 2} + V + H_{drive},$$

$$H^{(0)}_{T, k} =  \hbar \omega_{T, k} b_k^{\dagger}b_k + \hbar \frac{\delta_{T, k}}{2} b_k^{\dagger} b_k^{\dagger}b_k b_k,$$

$$V = J_C q_{T1} q_{T2},$$

$$\frac{H_{drive}}{h} = \left( \varepsilon_{T1} q_{T1} + \varepsilon_{T2} q_{T2} \right) \cos(2\pi f_d t).$$

The (approximate) charge operator for the transmon in terms of annihilation and creation operators reads

$$
q_{Tk} = i \biggl( \frac{E_{J,k}}{32 |\delta_{T,k} |} \biggr)^{1/4} (b_k^{\dagger} - b_k) = i q_{ZPF} (b_k^{\dagger} - b_k),
$$
where 
$$
E_{J, k} = \hbar \biggl(\frac{\omega_{T, k} - \delta_{T, k}}{8 | \delta_{T, k} |} \biggr)^2.
$$

In the following we introduce relaxation due to dielectric loss and pure dephasing on each qubit as error channels. 

The idea behind the cross-resonance gate is to drive one of the qubit at the frequency of the other using the fact that due to the coupling we have a $Z X$ term in the Hamiltonian. In particular, we will explore the idea of driving the fluxonium at the transmon frequency. The opposite idea to drive the transmon at the fluxonium frequency does not seem that promising. However, one might evaluate using Raman transitions.

# Introducing the qubits and the coupled system

In [2]:
with open('../flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [3]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "2"

#Transmon
levels_t = 3
transm_t = pysqkit.qubits.SimpleTransmon(
    label='T1', 
    max_freq=parameters_set[p_set]["max_freq_t"] , 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 4

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)
eigen_flx, eigstates_flx = flx.eig_states()

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cr_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm_t.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)

states_label = coupled_sys.all_state_labels()
states_dict = coupled_sys.states_as_dict(as_qobj=True)

In [4]:
state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp

The basis principle behind the gate is illustrated in the following figure

The figure shows the energy of the bare levels. The orange arrows indicate the non-zero matrix elements between the (bare) levels provided by the capacitive coupling $V$: the more shaded the arrow the weaker the coupling. 
We look at the following quantities : 

$$\zeta_{ZZ} = E_{00} + E_{11} - E_{01} - E_{10}$$
$$\mu_{YZ} = \lvert \langle 10 | q_{F} | 00 \rangle - \langle 11 | q_{F} |01 \rangle \rvert $$
$$\mu_{Y} = \lvert \langle 10 | q_{F} | 00 \rangle + \langle 11 | q_{F} |01 \rangle \rvert $$

The second term quantifies the gate speed. Notice that if the phase of the drive is $0$ one hase a $YZ$ term from the charge operator. 

For the given parameters we have

In [7]:
eps_test = 0.3
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*util_tf_cr.zz(coupled_sys))))
q_op = coupled_sys["F"].charge_op()
display(Latex(r'$\mu_{{YZ}} = {:.4f} \mathrm{{MHz}}$'.format(util_tf_cr.mu_yz_flx(comp_states, q_op, eps_test*1e3))))
display(Latex(r'$\mu_{{YI}} = {:.4f} \mathrm{{MHz}}$'.format(util_tf_cr.mu_yi_flx(comp_states, q_op, eps_test*1e3))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
f_03 = eigen_flx[3] - eigen_flx[0]
q_03 = get_mat_elem(flx.charge_op(), flx.state(0)[1], flx.state(3)[1])
f_12 = eigen_flx[2] - eigen_flx[1]
q_12 = get_mat_elem(flx.charge_op(), flx.state(1)[1], flx.state(2)[1])
eps = 0.3
freq_drive = transm_t.freq

In [9]:
util_tf_cr.mu_yz_flx(comp_states, coupled_sys['F'].charge_op(), eps*1e3)

-0.5040775655934453

In [11]:
util_tf_cr.mu_yz_flx_sw(transm_t, flx, jc, eps*1e3)

-0.5440270043026674

In [14]:
util_tf_cr.cr_gate_time(np.abs(util_tf_cr.mu_yz_flx_sw(transm_t, flx, jc, eps)))

229.76800602062895

# Generating data

In [17]:
def get_cr_effect(
    freq: float,
    eps: float,
    jc: float,
    flx: Qubit,
    eigen_flx: List[float],
    eigstates_flx: List[np.ndarray]
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    
    
    transm_t = pysqkit.qubits.SimpleTransmon(
            label='T_tmp', 
            max_freq=freq, 
            anharm=-0.3,
            dim_hilbert=3,
        )
    
    f_03 = eigen_flx[3] - eigen_flx[0]
    q_03 = get_mat_elem(flx.charge_op(), eigstates_flx[0], eigstates_flx[3])
    f_12 = eigen_flx[2] - eigen_flx[1]
    q_12 = get_mat_elem(flx.charge_op(), eigstates_flx[1], eigstates_flx[2])
    freq_drive = transm_t.freq
    min_delta = np.min([np.abs(freq_drive - f_03)/q_03, np.abs(freq_drive - f_12)/q_12])
    eps_ratio = eps*np.max([np.abs(q_03/(freq_drive - f_03)), np.abs(q_12/(freq_drive - f_12))])
    
    
    coupled_sys = transm_t.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    mu_cr = util_tf_cr.mu_yz_flx(comp_states, coupled_sys['F'].charge_op(), eps*1e3)
    
    mu_cr_sw = util_tf_cr.mu_yz_flx_sw(transm_t, flx, jc, eps*1e3)
    
    zz_coeff = util_tf_cr.zz(coupled_sys)*1e3
    
    output = {}
    output["freq_fixed"] = transm_t.freq
    output["freq"] = freq
    output["eps"] = eps
    output["eps_ratio"] = eps_ratio
    output["cr"] = np.abs(mu_cr)
    output["cr_sw"] = np.abs(mu_cr_sw)
    output["zz"] = zz_coeff
    output["gate_time"] = util_tf_cr.cr_gate_time(np.abs(mu_cr/1e3))
    output["gate_time_sw"] = util_tf_cr.cr_gate_time(np.abs(mu_cr_sw/1e3))
    
    output["jc"] = jc*1e3
    
    return output

func = partial(get_cr_effect, eps=eps, jc=jc, flx=flx, eigen_flx=eigen_flx, eigstates_flx=eigstates_flx)

In [19]:
start = time.time()

output_test = func(5.0)

end = time.time()

print("Computation time = {} s".format(end - start))



Computation time = 0.24015140533447266 s


In [20]:
output_test

{'freq_fixed': 5.0,
 'freq': 5.0,
 'eps': 0.3,
 'eps_ratio': 0.10779439345553458,
 'cr': 0.41981343032359647,
 'cr_sw': 0.45885498574159145,
 'zz': 0.11647912536272287,
 'gate_time': 297.7513127763653,
 'gate_time_sw': 272.4172208742109,
 'jc': 20.0}

In [33]:
delta_freq = 0.8
n_points = 200
ref_freq = 5.0
freq_list = list(np.linspace(ref_freq - delta_freq, ref_freq + delta_freq, n_points))

n_process = 1

start = time.time()

pool = multiprocessing.Pool(processes=n_process)

result = pool.map(func, freq_list)

pool.close()
pool.join()

end = time.time()

print("Computation time = {} s".format(end - start))

# Nonsensical behaviour with more than 1 process...

Computation time = 51.60290336608887 s


In [35]:
save = False # set to True to save
if save:
    with open('tmp/data_cr_transmon_fluxonium_linear.txt', 'w') as fp:
        json.dump(result, fp)